In [25]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10] # 10 x 10 inches plot

In [73]:
N = 512
S = 10 # number of slices / CT-images / scans
r = np.random.random((S, N, N))
ro  = 1e-5 + 1.24e-3*r - 2.83e-7*r*r + 2.79e-11*r*r*r
# c = np.empty_like(r)
c = (ro + 0.112) * 1.38e-6
K = np.zeros((S, N, N))

# тут не факт что все 1
l = 1 # dt
h = 1 # dx = dy = dz

# initial conditions
P_pp = np.zeros((S, N, N), dtype=np.float64) # previous previous t - 2
P_p  = np.zeros((S, N, N), dtype=np.float64) # previous          t - 1
P    = np.zeros((S, N, N), dtype=np.float64) # current           t

# sound source location
A = 0
B = N // 2
C = N // 2
ro[A, B, C] = 1

# sine wave sound frequency
f = 440.
t = 0.0

In [54]:
print(ro[A, B, C])

1.0


In [33]:
# r.dtype
# c.shape
# r.shape
# type(c)

# it = np.nditer(r[2:-2, 2:-2, 2:-2], flags=['multi_index'])
# while not it.finished:
#     print(it.multi_index[0], r[it.multi_index])
#     it.iternext()

In [1]:
# slow explicit n^3, c-style implementation v0.2
for i in range(2, S - 2):
    for j in range(2, N - 2):
        for k in range(2, N - 2):
            if ro[i, j, k] < 0.1 and i != A and j != B and k != C: # refl􏰂ecting condition | use numpy masking
                P[i, j, k] = 0
            else:
                K[i, j, k] = l / h * c[i, j, k]
                K2 = K[i, j, k] ** 2
                K2_3_ro = K2 / 3 / ro[i, j, k]

                # need some optimisation, with K2, K2_3_ro, чтобы считать меньше
                # K2. K2_3_ro - put outside for loops
                # Первое слагаемое можно тоже вынести outside for loops - там простое сложение
                # можно slices + sum вместо много + + + +
                P[i, j, k] = (2 - 7.5 * K2) * P_p[i, j, k] - P_pp[i, j, k] \
                    + 4/3 * K2 * (P_p[i + 1, j    , k    ] + \
                                  P_p[i - 1, j    , k    ] + \
                                  P_p[i    , j + 1, k    ] + \
                                  P_p[i    , j - 1, k    ] + \
                                  P_p[i    , j    , k + 1] + \
                                  P_p[i    , j    , k - 1] ) \
                    - K2 / 12 *  (P_p[i + 2, j    , k    ] + \
                                  P_p[i - 2, j    , k    ] + \
                                  P_p[i    , j + 2, k    ] + \
                                  P_p[i    , j - 2, k    ] + \
                                  P_p[i    , j    , k + 2] + \
                                  P_p[i    , j    , k - 2] ) \
                    - K2_3_ro * ((P_p[i + 1, j    , k    ] - P_p[i-1 , j  , k  ]) - (P_p[i + 2, j, k] - P_p[i - 2, j, k]) / 8) * (ro[i + 1, j, k] - ro[i - 1, j, k]) \
                    - K2_3_ro * ((P_p[i    , j + 1, k    ] - P_p[i   , j-1, k  ]) - (P_p[i, j + 2, k] - P_p[i, j - 2, k]) / 8) * (ro[i, j + 1, k] - ro[i, j - 1, k]) \
                    - K2_3_ro * ((P_p[i    , j    , k + 1] - P_p[i-1 , j  , k-1]) - (P_p[i, j, k + 2] - P_p[i, j, k - 2]) / 8) * (ro[i, j, k + 1] - ro[i, j, k - 1])
                t += 0.1
                P[A, B, C] = np.sin(2 * np.pi * f * t)
                P_pp = P_
# plt.imshow(P[0])
# print(P[0])

NameError: name 'S' is not defined

In [ ]:
ro[0]

In [47]:
K[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [48]:
# plt.hist(K.flatten(), bins=100)
K[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [60]:
c[0][0]

array([1.56029157e-07, 1.56088923e-07, 1.55046868e-07, 1.54948436e-07,
       1.54860127e-07, 1.56100348e-07, 1.55845210e-07, 1.55225622e-07,
       1.54920168e-07, 1.56046582e-07])

In [78]:
import timeit
import numpy as np
from distutils.version import StrictVersion
from six.moves import range

setup = """
import numpy as np
from six.moves import range
n = 1000
x = np.random.random(n)
y = np.empty_like(x)
"""

# These methods modify y so that all its values are equal to those in x.
methods = [
"for i in range(n): y[i] = x[i]",
"y[:] = x"
]
if StrictVersion(np.__version__) >= StrictVersion('1.7.0'):
    methods.append("np.copyto(y, x)")

# These methods create a new object y with values equal to those in x.
methods.extend(
"""
y = np.empty_like(x)
y[:] = x
#
y = np.empty(n)
y[:] = x
#
y = np.zeros_like(x)
y[:] = x
#
y = np.zeros(n)
y[:] = x
#
y = np.copy(x)
#
y = np.array(x)
#
y = 1*x
""".split('#\n')
)

niter = 10000

for method in methods:
    print(method.strip())
    print('Time =', timeit.timeit(method, setup=setup, number=niter))
    print()

for i in range(n): y[i] = x[i]
Time = 2.2633061820233706

y[:] = x
Time = 0.006901468004798517

np.copyto(y, x)
Time = 0.006254913983866572

y = np.empty_like(x)
y[:] = x
Time = 0.012216715986141935

y = np.empty(n)
y[:] = x
Time = 0.0205626429815311

y = np.zeros_like(x)
y[:] = x
Time = 0.05558887901133858

y = np.zeros(n)
y[:] = x
Time = 0.02236265200190246

y = np.copy(x)
Time = 0.020289150008466095

y = np.array(x)
Time = 0.00822795499698259

y = 1*x
Time = 0.01861786301014945



In [80]:
P_pp = np.array([0, 1, 2, 3])
P_p  = np.array([4, 4, 4, 4])
P    = np.array([0, 0, 0, 0])

In [81]:
P_pp = P_p
P_p = P

In [86]:
P[3] = 7

In [89]:
P_pp

array([4, 4, 4, 4])

In [88]:
P_p

array([0, 0, 0, 7])